# CustomLayer 定義

如果你想要自己根據論文公式刻出一個model，下面一有一個最基本的範例可供使用
PyTorch 在定義model會需要用到Parameter來打包可訓練參數
這樣在優化器指定要優化的參數時，才可以透過Module.parameter()這個函式來快速調參

而要注意的是當資料是以Batch的形式呈現，計算時要注意batch的軸不能消失
否則在訓練資料的過程中就會失去Batch Training的意義

In [2]:
import torch

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

class LinearLayer(torch.nn.Module):

    def __init__(self, in_feature, out_feature) -> None:
        super(LinearLayer, self).__init__()
        self.w1 = torch.nn.Parameter(torch.randn(size = (out_feature, in_feature), requires_grad=True))
        self.b1 = torch.nn.Parameter(torch.randn(size = (out_feature, )))

    def forward(self, x):
        x = x @ self.w1.T + self.b1
        return x

In [4]:
linear = LinearLayer(20, 10)

In [5]:
for parm in linear.parameters():
    print(parm.shape)

torch.Size([10, 20])
torch.Size([10])


In [6]:
x = torch.randn(size=(20,20))

In [7]:
temp = linear(x)

In [8]:
temp.shape

torch.Size([20, 10])

In [9]:
tempModel = torch.nn.Linear(20,10)

In [10]:
for parm in tempModel.parameters():
    print(parm.shape)

torch.Size([10, 20])
torch.Size([10])


In [12]:
x = torch.randn(size=(2, 10))
b = torch.randn(size=(10,))

In [13]:
print(x)
print(b)

tensor([[ 0.6079, -1.2496,  2.0597, -0.4728, -0.9358,  0.0396, -1.0480,  1.2702,
          1.3784,  0.6608],
        [-0.3934,  0.6642,  0.0392, -0.8562, -1.0865,  0.0327, -2.6049, -1.6452,
          0.1189,  0.0759]])
tensor([-1.3242, -2.2082,  1.0410, -1.4352,  0.6415,  0.8393,  0.3254,  0.9716,
        -0.9552, -1.1094])


In [14]:
print(x+1)

tensor([[-0.7164, -3.4577,  3.1006, -1.9080, -0.2944,  0.8789, -0.7226,  2.2418,
          0.4232, -0.4487],
        [-1.7177, -1.5440,  1.0802, -2.2915, -0.4450,  0.8721, -2.2794, -0.6736,
         -0.8363, -1.0335]])
